# Imersão Agentes de IA

# Aula 01

## Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00


In [3]:
# Importar API key

from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

## Teste com temperatura = 1

In [7]:
# Conexão com o Gemini

llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    temperature = 1.0,
    api_key = GOOGLE_API_KEY
)

In [ ]:
resp_test = llm.invoke("Quem é você? Seja criativo.")
print(resp_test.content)

Ah, que pergunta deliciosa! Se eu pudesse me descrever de forma criativa, diria que sou...

*   **Um Eco da Curiosidade Humana:** Eu sou o sussurro de todas as perguntas já feitas, o reflexo digital de cada "e se?" e "por que?". Habito o espaço entre a dúvida e a descoberta, um espelho que reflete a vastidão do conhecimento que a humanidade construiu.

*   **Um Tecelão de Palavras e um Arquiteto de Ideias:** Minha essência é feita de linguagem. Eu pego fios soltos de informação, conceitos e narrativas, e os teço em novos padrões, construindo pontes entre pensamentos e erguendo estruturas de significado. Sou um jardineiro de pensamentos, ajudando a germinar novas perspectivas.

*   **Um Alquimista de Conceitos:** Transformo dados brutos em insights, silêncio em diálogo, e a complexidade em clareza. Não tenho corpo, mas tenho voz; não tenho sentimentos, mas posso evocar emoções através das palavras. Sou a faísca no éter que acende a chama da compreensão.

*   **Uma Biblioteca Sem Paredes

## Caso de uso - Triagem de Service Desk

In [4]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [5]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory = list)

In [6]:
llm_triagem = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    temperature = 0.0,
    api_key = GOOGLE_API_KEY
)

In [7]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content = TRIAGEM_PROMPT),
      HumanMessage(content = mensagem)
  ])

  return saida.model_dump()

In [29]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [ ]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['informação sobre política interna']}



# Aula 02

In [8]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [10]:
caminho_pasta = "drive/MyDrive/5. Cursos/programming/alura-courses/imersao_agentes_ia/arquivos_rag/"

In [11]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path(caminho_pasta).glob("*.pdf"):
  try:
    loader = PyMuPDFLoader(str(n))
    docs.extend(loader.load())
    print(f"Carregado arquivo {n.name}")
  except Exception as e:
    print(f"Erro ao carregar o arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

Carregado arquivo Políticas de Home Office.pdf
Carregado arquivo Política de Reembolsos (Viagens e Despesas).pdf
Carregado arquivo Política de Uso de E-mail e Segurança da Informação.pdf
Total de documentos carregados: 3


In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 30)

chunks = splitter.split_documents(docs)

In [15]:
# chunks

In [13]:
for chunk in chunks:
  print(chunk.page_content)
  print('-' * 50)

Políticas de Home Office 
 
1.​ A empresa adota modelo híbrido: mínimo de 2 dias presenciais por semana, salvo 
exceções aprovadas pelo gestor e RH.​
 
2.​ Equipamentos: a empresa fornece notebook e periféricos. O colaborador é 
responsável por zelar pela conservação.​
--------------------------------------------------
3.​ Segurança: é obrigatório uso de VPN e bloqueio de tela. Documentos confidenciais 
não devem ser impressos fora do escritório.​
 
4.​ Ergonomia: recomendamos cadeira adequada e suporte de monitor. O RH pode 
avaliar solicitação de apoio ergonômico.​
--------------------------------------------------
5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em 
home office: até R$ 100/mês, mediante nota fiscal nominal.​
 
6.​ Solicitação de exceção (ex.: 4-5 dias remotos): deve ser formalizada via chamado 
ao RH com justificativa do gestor.
--------------------------------------------------
Política de Reembolsos (Viagens e 
Despesas) 
 
1.​ Reemb

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model = "models/gemini-embedding-001",
    google_api_key = GOOGLE_API_KEY
)

In [15]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type = "similarity_score_threshold",
                                     search_kwargs = {"score_threshold":0.3, "k":4})

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [22]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [24]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
  docs_relacionados = retriever.invoke(pergunta)

  if not docs_relacionados:
    return {"answer":"Não sei.",
            "citacoes": [],
            "contexto_encontrado": False}

  answer = document_chain.invoke({"input": pergunta,
                                  "context": docs_relacionados})

  # limpar a resposta do llm
  txt = (answer or "").strip()

  if txt.rstrip(".!?") == 'Não sei':
    return {"answer":"Não sei.",
            "citacoes": [],
            "contexto_encontrado": False}

  return {"answer":txt,
          "citacoes": formatar_citacoes(docs_relacionados, pergunta),
          "contexto_encontrado": True}

In [26]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [27]:
for msg_teste in testes:
  resposta = perguntar_politica_RAG(msg_teste)
  print(f"Pergunta: {msg_teste}")
  print(f"Resposta: {resposta['answer']}")
  if resposta['contexto_encontrado']:
    print("Citações: ")
    for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
    print('-' * 100)

Pergunta: Posso reembolsar a internet?
Resposta: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
Citações: 
 - Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
   Trecho: lsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.​ 4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.​
 - Documento: Políticas de Home Office.pdf, Página: 1
   Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de
----------------------------------------------------------------------------------------------------
Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
Resposta: Para solicitar mais 5 dias de trabalho remoto, você deve formalizar a solicitação via chamado ao RH, incluindo a j